# Assignment 1
You should submit the **UniversityNumber.ipynb** file and your final prediction file **UniversityNumber.test.txt** to moodle. Make sure your code does not use your local files and that the results are reproducible. Before submitting, please **1. clean all outputs and 2. run all cells in your notebook and keep all running logs** so that we can check.

In [1]:
!pip install numpy==1.26.4 nltk==3.9.1 scikit-learn==1.6.1 gensim==4.3.3 setuptools

In [2]:
# Environment Setup and Validation
import sys
print(f"Python version: {sys.version}")

# Required package versions
required_packages = {
    'numpy': '1.26.4',
    'nltk': '3.9.1',
    'scikit-learn': '1.6.1',
    'gensim': '4.3.3'
}

def validate_environment():
    import pkg_resources
    installed = {pkg.key: pkg.version for pkg in pkg_resources.working_set}
    for package, min_version in required_packages.items():
        if package not in installed:
            print(f"❌ {package} not found!")
            return False
        if pkg_resources.parse_version(installed[package]) < pkg_resources.parse_version(min_version):
            print(f"❌ {package} version {installed[package]} is below minimum {min_version}")
            return False
        print(f"✓ {package} version {installed[package]}")
    return True

assert validate_environment(), "Please install required packages with correct versions"

Python version: 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]


<ipython-input-2-56c000555d47>:14: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


✓ numpy version 1.26.4
✓ nltk version 3.9.1
✓ scikit-learn version 1.6.1
✓ gensim version 4.3.3


# 1 $n$-gram Language Model

In [3]:
!mkdir -p data/lm
!wget -O data/lm/train.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2025/main/assignments/A1/data/lm/train.txt
!wget -O data/lm/dev.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2025/main/assignments/A1/data/lm/dev.txt
!wget -O data/lm/test.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2025/main/assignments/A1/data/lm/test.txt

--2025-02-28 14:11:09--  https://raw.githubusercontent.com/ranpox/comp3361-spring2025/main/assignments/A1/data/lm/train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8441444 (8.0M) [text/plain]
Saving to: ‘data/lm/train.txt’

data/lm/train.txt   100%[===================>]   8.05M  45.5MB/s    in 0.2s    

2025-02-28 14:11:09 (45.5 MB/s) - ‘data/lm/train.txt’ saved [8441444/8441444]

--2025-02-28 14:11:10--  https://raw.githubusercontent.com/ranpox/comp3361-spring2025/main/assignments/A1/data/lm/dev.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting r

## 1.1 Building vocabulary

### Instructions:
1. Implement a Vocabulary class that:
   - Handles OOV words (tokens occurring < 3 times)
   - Includes special tokens \<UNK>, \<START>, \<END>
   - Provides word2idx and idx2word mappings
2. Expected vocabulary size: 24,067 words

### Code

In [4]:
# 1.1 Building vocabulary
import nltk
from collections import Counter
from typing import List, Dict

class Vocabulary:
    def __init__(self):
        self.word2idx = {'<UNK>': 0, '<START>': 1, '<END>': 2}
        self.idx2word = ['<UNK>', '<START>', '<END>']
        self.word_counts = {}

    def build_vocab(self, filepath: str, min_freq: int = 3) -> None:
        """Build vocabulary from training file

        Args:
            filepath: Path to training file
            min_freq: Minimum frequency threshold for words
        """
        # TODO: Implement vocabulary building
        with open(filepath, 'r', encoding='utf-8') as file:
            text = file.read().lower()
            words = nltk.word_tokenize(text)

        self.word_counts = Counter(words)

        for word, count in self.word_counts.items():
            if count >= min_freq and word not in self.word2idx:
                idx = len(self.word2idx)
                self.word2idx[word] = idx
                self.idx2word.append(word)

    def __len__(self) -> int:
        return len(self.word2idx)

    def get_id(self, word: str) -> int:
        """Get ID for word (returns UNK if not in vocab)"""
        # TODO: Implement
        return self.word2idx.get(word, self.word2idx['<UNK>'])

    def get_word(self, idx: int) -> str:
        """Get word for ID"""
        # TODO: Implement
        if idx < len(self.idx2word):
            return self.idx2word[idx]
        return '<UNK>'

# Test vocabulary
nltk.download('punkt_tab')
vocab = Vocabulary()
vocab.build_vocab('data/lm/train.txt')
print(f"Vocabulary size: {len(vocab)}")
assert len(vocab) == 24067, "Vocabulary size should be 24,067"

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Vocabulary size: 24067


### Discussion

Discuss the number of parameters in n-gram models here




In n-gram models, the number of parameters grows exponentially with the vocabulary size $|V|$ and the n-gram order $n$, as it equals $|V|^n$.

This is because the model must store the probability of each word given its context (the previous $n-1$ words). For example, in a trigram model with the vocabulary we just built, we need a 3D array ``prob[word][context_1][context_2]``. Since each dimension has size $|V|$, the total size of the array is $24,067^3$, which is an extremely large number.

## 1.2 $n$-gram Language Modeling

### Instructions:
1. Implement an n-gram language model that:
   - Counts n-grams from training data
   - Computes probabilities
   - Calculates perplexity
2. Test with both unigram and bigram models
3. Report perplexity on train and dev sets

### Code

In [10]:
import numpy as np
from collections import defaultdict
from typing import List, Tuple

class LanguageModel:
    def __init__(self, vocab: Vocabulary, n: int):
        """Initialize n-gram language model

        Args:
            vocab: Vocabulary object
            n: Order of n-gram model
        """
        self.vocab = vocab
        self.n = n
        self.ngram_counts = defaultdict(int)
        self.context_counts = defaultdict(int)

    def get_ngrams(self, tokens: List[str], n: int) -> List[Tuple[str]]:
        """Extract n-grams from token sequence"""
        # TODO: Implement n-gram extraction
        ngrams = []
        for i in range(len(tokens) - n + 1):
            ngrams.append(tuple(tokens[i:i+n]))
        return ngrams

    def train(self, filepath: str) -> None:
        """Train n-gram language model"""
        # TODO: Implement training
        with open(filepath, 'r', encoding='utf-8') as file:
            for line in file:
                tokens = nltk.word_tokenize(line.lower())
                tokens = [self.vocab.idx2word[self.vocab.get_id(token)] for token in tokens]
                num_start = self.n - 1 if self.n > 1 else 1
                tokens = ['<START>'] * num_start + tokens + ['<END>']

                ngrams = self.get_ngrams(tokens, self.n)
                contexts = self.get_ngrams(tokens, self.n - 1)

                for ngram in ngrams:
                    self.ngram_counts[ngram] += 1
                for context in contexts:
                    self.context_counts[context] += 1


    def get_prob(self, word: str, context: Tuple[str]) -> float:
        """Get probability of word given context"""
        # TODO: Implement probability calculation
        ngram = context + (word,)
        if self.context_counts[context] == 0:
            return 0
        return self.ngram_counts[ngram] / self.context_counts[context]

    def perplexity(self, filepath: str) -> float:
        """Calculate perplexity on given text"""
        # TODO: Implement perplexity calculation
        with open(filepath, 'r', encoding='utf-8') as file:
            log_prob_sum = 0
            num_words = 0
            for line in file:
                tokens = nltk.word_tokenize(line.lower())
                tokens = [self.vocab.idx2word[self.vocab.get_id(token)] for token in tokens]
                num_start = self.n - 1 if self.n > 1 else 1
                tokens = ['<START>'] * (self.n - 1) + tokens + ['<END>']

                for i in range(len(tokens) - self.n + 1):
                    ngram = tuple(tokens[i:i+self.n])
                    context = tuple(tokens[i:i+self.n-1])
                    word = ngram[-1]

                    prob = self.get_prob(word, context)
                    if prob == 0:
                        prob = 1e-10
                    log_prob_sum += np.log2(prob)
                    num_words += 1

        perplexity = 2 ** (-log_prob_sum / num_words)
        return perplexity


# Test language model
lm_unigram = LanguageModel(vocab, n=1)
lm_unigram.train('data/lm/train.txt')
train_perplexity = lm_unigram.perplexity('data/lm/train.txt')
dev_perplexity = lm_unigram.perplexity('data/lm/dev.txt')
print(f"Perplexity on train (unigram): {train_perplexity:.2f}")
print(f"Perplexity on dev (unigram): {dev_perplexity:.2f}")

lm_bigram = LanguageModel(vocab, n=2)
lm_bigram.train('data/lm/train.txt')
train_perplexity = lm_bigram.perplexity('data/lm/train.txt')
dev_perplexity = lm_bigram.perplexity('data/lm/dev.txt')
print(f"Perplexity on train (bigram): {train_perplexity:.2f}")
print(f"Perplexity on dev (bigram): {dev_perplexity:.2f}")

assert dev_perplexity > 0, "Perplexity should be positive"

Perplexity on train (unigram): 943.86
Perplexity on dev (unigram): 879.03
Perplexity on train (bigram): 81.47
Perplexity on dev (bigram): 3124.34


### Discussion

Compare unigram and bigram model performance here

The unigram and bigram model performance are as follows:

|model|perplexity (train set)|perplexity (dev set)|
|---|---|---|
|unigram|943.86|879.03|
|bigram|81.47|3124.34|

On the training set, the bigram model achieves a significantly lower perplexity compared to the unigram model. This is because the unigram model does not rely on any contextual information.



On the dev set, both models exhibit poor perplexity, with the bigram model performing significantly worse than the unigram model, which is unexpected. This is likely due to the presence of unknown contexts (e.g., unseen bigrams) leads to zero probabilities, and since $\log_2(0) \to -\infty$, this significantly increases perplexity. This effect is more pronounced in the bigram model due to its reliance on context.

## 1.3 Smoothing

### 1.3.1 Add-one (Laplace) smoothing

### Code

In [11]:
class AddOneLanguageModel(LanguageModel):
    def get_prob(self, word: str, context: Tuple[str]) -> float:
        """Get smoothed probability using add-one smoothing"""
        # TODO: Implement add-one smoothing
        ngram = context + (word,)
        context_count = self.context_counts[context]
        ngram_count = self.ngram_counts[ngram]
        vocab_size = len(self.vocab)

        return (ngram_count + 1) / (context_count + vocab_size)

# Test add-one smoothing
add_one_lm = AddOneLanguageModel(vocab, n=2)
add_one_lm.train('data/lm/train.txt')
print(f"Add-one perplexity on train: {add_one_lm.perplexity('data/lm/train.txt'):.2f}")
print(f"Add-one perplexity on dev: {add_one_lm.perplexity('data/lm/dev.txt'):.2f}")

Add-one perplexity on train: 1291.30
Add-one perplexity on dev: 1512.07


#### Discussion

Analyze how add-one smoothing affects the model performance

The bigram model with add-one smoothing performance is as follows:

|dataset|perplexity|
|---|---|
|training set|1291.30|
|dev set|1512.07|

The bigram model with add-one smoothing significantly reduces perplexity on the dev set but increases perplexity on the training set.

On the dev set, previously unseen contexts are now assigned a probability, preventing them from contributing as heavily to perplexity as they did before.

However, on the training set, this smoothing dilutes the original probabilities of known contexts, leading to an increase in perplexity. This tradeoff can be viewed as the cost for the bigram model to achieve better generalization.

### 1.3.2 Add-k smoothing

#### Instructions:
1. Implement add-k smoothing with configurable k
2. Try k values: 0.1, 0.5, 1.0

#### Code

In [12]:
class AddKLanguageModel(LanguageModel):
    def __init__(self, vocab: Vocabulary, n: int, k: float):
        super().__init__(vocab, n)
        self.k = k

    def get_prob(self, word: str, context: Tuple[str]) -> float:
        """Get smoothed probability using add-k smoothing"""
        # TODO: Implement add-k smoothing
        ngram = context + (word,)
        context_count = self.context_counts[context]
        ngram_count = self.ngram_counts[ngram]
        vocab_size = len(self.vocab)

        return (ngram_count + k) / (context_count + k * vocab_size)

# Test add-k smoothing with different k values
k_values = [0.01, 0.1, 0.5, 1.0]
for k in k_values:
    add_k_lm = AddKLanguageModel(vocab, n=2, k=k)
    add_k_lm.train('data/lm/train.txt')
    print(f"Add-{k} perplexity on train: {add_k_lm.perplexity('data/lm/train.txt'):.2f}")
    print(f"Add-{k} perplexity on dev: {add_k_lm.perplexity('data/lm/dev.txt'):.2f}")

Add-0.01 perplexity on train: 154.54
Add-0.01 perplexity on dev: 421.68
Add-0.1 perplexity on train: 379.19
Add-0.1 perplexity on dev: 648.66
Add-0.5 perplexity on train: 878.09
Add-0.5 perplexity on dev: 1130.63
Add-1.0 perplexity on train: 1291.30
Add-1.0 perplexity on dev: 1512.07


#### Discussion

Compare performance with different k values

The bigram model with add-k smoothing performance is as follows:

|dataset|k=0.01|k=0.1|k=0.5|k=1.0 (add-1 smoothing)|
|---|---|---|---|---|
|training set|186.75|474.68|1132.34|1682.19|
|dev set|530.65|829.50|1472.11|1981.10|

We can see that as $k$ decreases, the perplexity also decreases.

This result shows that add-one smoothing over-smooths and performs worse. Smaller $k$ values better balance fitting training data and generalizing to unseen data, while larger $k$ values dilute known bigram probabilities too aggressively, increasing perplexity on both sets.

### 1.3.3 Linear Interpolation

#### Instructions:
1. Implement linear interpolation of unigram, bigram, and trigram models
2. Initial lambdas: [0.1, 0.3, 0.6]

#### Code

In [18]:
class InterpolatedLanguageModel:
    def __init__(self, vocab: Vocabulary, lambdas: List[float]):
        """
        Args:
            vocab: Vocabulary object
            lambdas: List of interpolation weights (should sum to 1)
        """
        assert abs(sum(lambdas) - 1.0) < 1e-6, "Lambdas must sum to 1"
        self.vocab = vocab
        self.lambdas = lambdas
        self.max_n = len(self.lambdas)
        self.models = [LanguageModel(vocab, n=i+1) for i in range(len(lambdas))]

    def train(self, filepath: str) -> None:
        """Train all n-gram models"""
        for model in self.models:
            model.train(filepath)

    def EM_optimize(self, filepath: str, max_iter: int = 10, tol: float = 1e-6) -> None:
        """Optimize hyperparameters on dev dataset"""
        with open(filepath, 'r', encoding='utf-8') as file:
            ngrams = []
            for line in file:
                tokenized_line = nltk.word_tokenize(line.lower())
                tokenized_line = [self.vocab.idx2word[self.vocab.get_id(token)] for token in tokenized_line]
                num_start = self.max_n - 1 if self.max_n > 1 else 1
                tokenized_line = ['<START>'] * num_start + tokenized_line + ['<END>']
                ngrams.extend([tuple(tokenized_line[i:i+self.max_n]) for i in range(len(tokenized_line)-self.max_n+1)])

        for iteration in range(max_iter):
            gamma = np.zeros((len(ngrams), len(self.lambdas)))
            for idx, ngram in enumerate(ngrams):
                context = ngram[:-1]
                word = ngram[-1]
                for i, model in enumerate(self.models):
                    model_context = context[-(model.n - 1):] if model.n > 1 else ()
                    gamma[idx, i] = self.lambdas[i] * model.get_prob(word, model_context)
                gamma[idx, :] /= np.sum(gamma[idx, :])

            new_lambdas = np.sum(gamma, axis=0) / len(ngrams)

            if np.max(np.abs(new_lambdas - self.lambdas)) < tol:
                break

            self.lambdas = new_lambdas


    def get_prob(self, word: str, context: Tuple[str]) -> float:
        """Get interpolated probability"""
        # TODO: Implement interpolated probability
        prob = 0.0
        for weight, model in zip(self.lambdas, self.models):
            model_context = context[-(model.n - 1):] if model.n > 1 else ()
            prob += weight * model.get_prob(word, model_context)
        return prob

    def perplexity(self, filepath: str) -> float:
        """Calculate perplexity using interpolated probabilities"""
        # TODO: Implement perplexity calculation

        log_prob_sum = 0
        num_words = 0
        with open(filepath, 'r', encoding='utf-8') as file:
            for line in file:
                tokens = nltk.word_tokenize(line.lower())
                tokens = [self.vocab.idx2word[self.vocab.get_id(token)] for token in tokens]
                num_start = self.max_n - 1 if self.max_n > 1 else 1
                tokens = ['<START>'] * num_start + tokens + ['<END>']

                for i in range(len(tokens) - self.max_n + 1):
                    ngram = tuple(tokens[i:i+self.max_n])
                    context = tuple(tokens[i:i+self.max_n-1])
                    word = ngram[-1]

                    prob = self.get_prob(word, context)
                    if prob == 0:
                        prob = 1e-10
                    log_prob_sum += np.log2(prob)
                    num_words += 1

        perplexity = 2 ** (-log_prob_sum / num_words)
        return perplexity


# Test interpolated model
initial_lambdas = [0.2, 0.3, 0.5]  # Example weights for unigram, bigram, trigram
interpolated_lm = InterpolatedLanguageModel(vocab, initial_lambdas)
interpolated_lm.train('data/lm/train.txt')
interpolated_lm_with_em = InterpolatedLanguageModel(vocab, initial_lambdas)
interpolated_lm_with_em.train('data/lm/train.txt')
interpolated_lm_with_em.EM_optimize('data/lm/dev.txt')


print(f"Interpolated perplexity on train: {interpolated_lm.perplexity('data/lm/train.txt'):.2f}")
print(f"Interpolated perplexity on dev: {interpolated_lm.perplexity('data/lm/dev.txt'):.2f}")
print(f"Interpolated perplexity on test: {interpolated_lm.perplexity('data/lm/test.txt'):.2f}")

print("")
print(f"Best lambdas found by EM algorithm: ", interpolated_lm_with_em.lambdas)
print(f"Interpolated with EM optimization perplexity on train: {interpolated_lm_with_em.perplexity('data/lm/train.txt'):.2f}")
print(f"Interpolated with EM optimization perplexity on dev: {interpolated_lm_with_em.perplexity('data/lm/dev.txt'):.2f}")
print(f"Interpolated with EM optimization perplexity on test: {interpolated_lm_with_em.perplexity('data/lm/test.txt'):.2f}")

Interpolated perplexity on train: 13.53
Interpolated perplexity on dev: 302.56
Interpolated perplexity on test: 307.37

Best lambdas found by EM algorithm:  [0.29649361 0.56091062 0.14259577]
Interpolated with EM optimization perplexity on train: 27.91
Interpolated with EM optimization perplexity on dev: 259.87
Interpolated with EM optimization perplexity on test: 263.59


#### Discussion

Analyze the effectiveness of interpolation

For the initial hyperparameters $\lambda_1=0.2, \lambda_2=0.3,\lambda_3=0.5$, the results are:

|training set|dev set|test set|
|---|---|---|
|13.33|367.98|374.16|


This result is better than all smoothing methods we have tried before, which proves the effectivenss of interpolation.

The hyperparameters I optimized on dev set are:
$$
\lambda_1=0.326, \lambda_2=0.543, \lambda_3=0.131
$$
The corresponding results for this set of parameters are:

|training set|dev set|test set|
|---|---|---|
|30.54|307.84|312.40|

This set of hyperparameters further decreases perplexity on dev set, and also achieve better results on test set.

The optimization algorithm I used is EM algorithm, and I will discuss about it in the following section (1.3.4 Optimization).


### 1.3.4 Optimization

#### Discussion

I discovered online that the EM algorithm is an efficient method for optimizing the parameters of interpolation in statistical models.

The Expectation-Maximization (EM) algorithm is an iterative method for estimating parameters in models with missing or unobserved data. It alternates between computing expected values for the latent variables (E-Step) and updating parameters to maximize the likelihood of the observed data (M-Step), repeating until convergence.

I also implemented this algorithm (the `EM_optimize` function) and used it for optimizing the hyperparameters on dev set. The results are quite promising, as shown above.

# 2 Word Vectors

In [ ]:
def load_embedding_model():
    """ Load GloVe Vectors
        Return:
            wv_from_bin: All 400000 embeddings, each lengh 50
    """
    import gensim.downloader as api
    wv_from_bin = api.load("glove-wiki-gigaword-200")
    print("Loaded vocab size %i" % len(list(wv_from_bin.index_to_key)))
    return wv_from_bin

wv_from_bin = load_embedding_model()

Loaded vocab size 400000


## 2.1 Find most similar word
### Instructions:
1. Implement function to find similar words using cosine similarity
2. Expected similarity scores should be between 0 and 1
3. Return top 3 similar words for each query

In [ ]:
def find_most_similar(word: str, wv_from_bin) -> List[Tuple[str, float]]:
    """Find most similar words using cosine similarity

    Args:
        word: Query word
        wv_from_bin: Loaded word vectors

    Returns:
        List of (word, similarity) tuples
    """
    # TODO: Implement similarity search
    try:
        similar_words = wv_from_bin.most_similar(word, topn=3)
        return similar_words
    except KeyError:
        return []

test_words = [
    'ubiquitous',
    'serendipity',
    'melancholy',
    'paradox',
    'ethereal',
    'cacophony'
]
for word in test_words:
    similar_words = find_most_similar(word, wv_from_bin)
    print(f"\nMost similar to '{word}':")
    for similar_word, similarity in similar_words:
        print(f"{similar_word}: {similarity:.3f}")
        assert 0 <= similarity <= 1, f"Invalid similarity score: {similarity}"


Most similar to 'ubiquitous':
omnipresent: 0.606
commonplace: 0.574
everywhere: 0.516

Most similar to 'serendipity':
happenstance: 0.614
serendipitous: 0.450
silliness: 0.440

Most similar to 'melancholy':
melancholic: 0.776
wistful: 0.749
introspective: 0.691

Most similar to 'paradox':
paradoxes: 0.640
contradiction: 0.503
anomaly: 0.494

Most similar to 'ethereal':
otherworldly: 0.699
dreamy: 0.675
seductive: 0.613

Most similar to 'cacophony':
deafening: 0.644
cacophonous: 0.606
shrill: 0.591


## 2.2 Finding Analogies
Use vector addition and subtraction to compute target vectors for the analogies below. After computing each target vector, find the top three candidates by cosine similarity. Report the candidates and their similarities to the target vector.

- dog : puppy :: cat : ?:
- speak : speaker :: sing : ?:
- France : French :: England : ?:
- France : wine :: England : ?

In [ ]:
def find_analogy(a: str, b: str, c: str, wv_from_bin) -> List[Tuple[str, float]]:
    """Find word analogies using vector arithmetic

    Args:
        a, b, c: Words for analogy a:b :: c:?
        wv_from_bin: Loaded word vectors

    Returns:
        List of (word, similarity) tuples
    """
    # TODO: Implement analogy finding
    try:
        results = wv_from_bin.most_similar(positive=[b, c], negative=[a], topn=3)
        return results
    except KeyError as e:
        return []

# Test analogies
analogies = [
    ('king', 'queen', 'man'),
    ('paris', 'france', 'rome'),
    ('france', 'french', 'germany'),
    ('london', 'england', 'paris'),
    ('cat', 'kitten', 'dog'),
    ('rich', 'poor', 'strong')
]
for a, b, c in analogies:
    print(f"\nAnalogy: {a}:{b} :: {c}:?")
    results = find_analogy(a, b, c, wv_from_bin)
    for word, similarity in results:
        print(f"{word}: {similarity:.3f}")
        assert 0 <= similarity <= 1, f"Invalid similarity score: {similarity}"


Analogy: king:queen :: man:?
woman: 0.725
girl: 0.589
she: 0.571

Analogy: paris:france :: rome:?
italy: 0.773
spain: 0.611
italian: 0.566

Analogy: france:french :: germany:?
german: 0.940
austrian: 0.699
berlin: 0.652

Analogy: london:england :: paris:?
france: 0.712
french: 0.578
marseille: 0.555

Analogy: cat:kitten :: dog:?
puppy: 0.615
puppies: 0.499
rottweiler: 0.446

Analogy: rich:poor :: strong:?
weak: 0.655
despite: 0.629
stronger: 0.573


# 3 Sentiment analysis

In [ ]:
!mkdir -p data/classification
!wget -O data/classification/train.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2025/main/assignments/A1/data/classification/train.txt
!wget -O data/classification/dev.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2025/main/assignments/A1/data/classification/dev.txt
!wget -O data/classification/test-blind.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2025/main/assignments/A1/data/classification/test-blind.txt

--2025-02-26 09:37:45--  https://raw.githubusercontent.com/ranpox/comp3361-spring2025/main/assignments/A1/data/classification/train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 738844 (722K) [text/plain]
Saving to: ‘data/classification/train.txt’

data/classification 100%[===================>] 721.53K  --.-KB/s    in 0.008s  

2025-02-26 09:37:45 (91.4 MB/s) - ‘data/classification/train.txt’ saved [738844/738844]

--2025-02-26 09:37:45--  https://raw.githubusercontent.com/ranpox/comp3361-spring2025/main/assignments/A1/data/classification/dev.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:44

## 3.1 Logistic Regression

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import numpy as np

def load_data(filepath: str) -> Tuple[List[str], List[int]]:
    """Load text and labels from file"""
    texts, labels = [], []
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip().split('\t')
            if len(line) > 1:
                label, text = line
                texts.append(text)
                labels.append(int(label))
            else:
                texts.extend(line)
    return texts, labels

class SentimentClassifier:
    def __init__(self, feature_type: str = 'unigram'):
        """
        Args:
            feature_type: One of 'unigram', 'bigram', or 'glove'
        """
        self.feature_type = feature_type
        self.vectorizer = None
        self.classifier = LogisticRegression(random_state=42)
        self.wv_from_bin = wv_from_bin

    def extract_features(self, texts: List[str]) -> np.ndarray:
        """Extract features based on feature_type"""
        if self.feature_type == 'unigram':
            # TODO: Implement unigram feature extraction
            if self.vectorizer is None:
                self.vectorizer = CountVectorizer(ngram_range=(1, 1))
                return self.vectorizer.fit_transform(texts).toarray()
            else:
                return self.vectorizer.transform(texts).toarray()
        elif self.feature_type == 'bigram':
            # TODO: Implement bigram feature extraction
            if self.vectorizer is None:
                self.vectorizer = CountVectorizer(ngram_range=(2, 2))
                return self.vectorizer.fit_transform(texts).toarray()
            else:
                return self.vectorizer.transform(texts).toarray()
        elif self.feature_type == 'glove':
            # TODO: Implement glove feature extraction
            # Average word vectors for each text
            features = []
            for text in texts:
                words = text.split()
                word_vectors = [self.wv_from_bin[word] for word in words if word in self.wv_from_bin]

                if len(word_vectors) > 0:
                    features.append(np.mean(word_vectors, axis=0))
                else:
                    features.append(np.zeros(200))
            return np.array(features)
        else:
            raise ValueError(f"Invalid feature type: {self.feature_type}")

    def train(self, train_texts: List[str], train_labels: List[int]) -> None:
        """Train sentiment classifier"""
        # TODO: Implement training
        features = self.extract_features(train_texts)
        self.classifier.fit(features, train_labels)

    def predict(self, texts: List[str]) -> np.ndarray:
        """Predict sentiment labels"""
        # TODO: Implement prediction
        features = self.extract_features(texts)
        return self.classifier.predict(features)

    def evaluate(self, texts: List[str], labels: List[int]) -> Dict:
        """Evaluate classifier performance"""
        predictions = self.predict(texts)
        return classification_report(labels, predictions, output_dict=True)

# Train and evaluate models with different features
train_texts, train_labels = load_data('data/classification/train.txt')
dev_texts, dev_labels = load_data('data/classification/dev.txt')

results = {}
for feature_type in ['unigram', 'bigram', 'glove']:
    classifier = SentimentClassifier(feature_type)
    classifier.train(train_texts, train_labels)
    results[feature_type] = classifier.evaluate(dev_texts, dev_labels)

# Print results
for feature_type, metrics in results.items():
    print(f"\n{feature_type.upper()} Results:")
    print(f"Precision: {metrics['weighted avg']['precision']:.3f}")
    print(f"Recall: {metrics['weighted avg']['recall']:.3f}")
    print(f"F1-score: {metrics['weighted avg']['f1-score']:.3f}")


UNIGRAM Results:
Precision: 0.789
Recall: 0.789
F1-score: 0.789

BIGRAM Results:
Precision: 0.721
Recall: 0.718
F1-score: 0.716

GLOVE Results:
Precision: 0.769
Recall: 0.768
F1-score: 0.768


Compare the performance of three types of features on dev set. Report the weighted average precision, recall and F1-score for each feature set.

| Feature | precision | recall | F1-score |
| ----------- | --------- | ------ | -------- |
| unigram     |  0.789    |   0.789  |    0.789  |
| bigram      |  0.721      | 0.718    |   0.716    |
| GloVe       |      0.769  |   0.768   |   0.768    |

The results suggest that unigram features perform the best in terms of precision, recall, and F1-score, indicating that simple word-level features capture the sentiment well in this task.

In comparison, bigram features show a slight drop in performance, which could be due to the sparsity of meaningful bigrams in the dataset or the introduction of noise, making it harder for the model to generalize.

Finally, the GloVe-based model performs similarly to unigrams, which might be because the averaging of word embeddings doesn't capture enough contextual information for significant improvement over unigram features.

## 3.2 Better Feature

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import numpy as np
from typing import List, Tuple, Dict
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, trainers

def load_data(filepath: str) -> Tuple[List[str], List[int]]:
    """Load text and labels from file"""
    texts, labels = [], []
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip().split('\t')
            if len(line) > 1:
                label, text = line
                texts.append(text)
                labels.append(int(label))
            else:
                texts.extend(line)
    return texts, labels

class SentimentClassifier:
    def __init__(self):
        self.tokenizer = None
        self.vectorizer = None
        self.classifier = LogisticRegression(random_state=42)

    def train_bpe_tokenizer(self, texts: List[str], vocab_size: int = 10000):
        tokenizer = Tokenizer(models.BPE())
        tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
        tokenizer.decoder = decoders.BPEDecoder()

        trainer = trainers.BpeTrainer(vocab_size=vocab_size)
        tokenizer.train_from_iterator(texts, trainer=trainer)
        self.tokenizer = tokenizer

    def extract_features(self, texts: List[str]) -> np.ndarray:
        """Extract features using BPE tokenization"""
        if self.tokenizer is None:
            self.train_bpe_tokenizer(texts)

        # Tokenize texts
        tokenized_texts = [self.tokenizer.encode(text).tokens for text in texts]

        # Convert tokens to space-separated strings for CountVectorizer
        tokenized_texts = [" ".join(tokens) for tokens in tokenized_texts]

        if self.vectorizer is None:
            self.vectorizer = CountVectorizer(ngram_range=(1, 1), max_df=500)
            return self.vectorizer.fit_transform(tokenized_texts).toarray()
        else:
            return self.vectorizer.transform(tokenized_texts).toarray()

    def train(self, train_texts: List[str], train_labels: List[int]) -> None:
        """Train sentiment classifier"""
        features = self.extract_features(train_texts)
        self.classifier.fit(features, train_labels)

    def predict(self, texts: List[str]) -> np.ndarray:
        """Predict sentiment labels"""
        features = self.extract_features(texts)
        return self.classifier.predict(features)

    def evaluate(self, texts: List[str], labels: List[int]) -> Dict:
        """Evaluate classifier performance"""
        predictions = self.predict(texts)
        return classification_report(labels, predictions, output_dict=True)

# Train and evaluate models with different features
train_texts, train_labels = load_data('data/classification/train.txt')
dev_texts, dev_labels = load_data('data/classification/dev.txt')

results = {}
classifier = SentimentClassifier()
classifier.train(train_texts, train_labels)
results = classifier.evaluate(dev_texts, dev_labels)

# Print results
print(f"Precision: {results['weighted avg']['precision']:.3f}")
print(f"Recall: {results['weighted avg']['recall']:.3f}")
print(f"F1-score: {results['weighted avg']['f1-score']:.3f}")

Precision: 0.805
Recall: 0.805
F1-score: 0.805
